In [2]:
import pandas as pd
print(pd.DataFrame([['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]))

    0   1     2     3
0  I1  I2    I5  None
1  I2  I4  None  None
2  I2  I3  None  None
3  I1  I2    I4  None
4  I1  I3  None  None
5  I2  I3  None  None
6  I1  I3  None  None
7  I1  I2    I3    I5
8  I1  I2    I3  None


In [19]:
import pandas as pd
'''
Load data
'''
def loadSimpDat():
    simpDat = [['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]
    '''simpDat=[]
    with open('Example_data.txt', 'r') as f:
        for line in f.readlines():
            L=line.split(',')
            L[-1]=L[-1].replace('\n','')
            simpDat.append(L)'''
    return simpDat

#tree node
class TreeNode:
    def __init__(self, item, count, parentNode):
        self.item=item
        self.count=count
        self.nodelink=None
        self.parent=parentNode
        self.children={}
    def inc_count(self,num):
        self.count+=num

'''
Generate Headertable and remove item does not meet minimum support
|item|counts|Nodelink
'''
def GenC1(trans,min_support):
    C1={}
    for i in trans:
        for ii in set(i):
            if ii not in C1:
                try:
                    C1[ii]=trans[i]
                except:
                    C1[ii]=1
            else:
                try:
                    C1[ii]+=trans[i]
                except:
                    C1[ii]+=1
    C1={key:value for key, value in C1.items() if value>=min_support}#include min_support
    return {i:[k,None] for i,k in sorted(C1.items(),key=lambda x:x[1],reverse=True)}

def link_same_items(curr,targetNode):
    while curr.nodelink is not None:
        curr=curr.nodelink
    curr.nodelink=targetNode


'''
Build Tree
'''
def link_same_items(curr,targetNode):
    while curr.nodelink is not None:
        curr=curr.nodelink
    curr.nodelink=targetNode

def updateTree_Table(SortedItems,FPTree,headertable,count):
    if SortedItems[0] in FPTree.children:
        FPTree.children[SortedItems[0]].inc_count(count)
    else:
        FPTree.children[SortedItems[0]]=TreeNode(SortedItems[0],count,FPTree)
        if headertable[SortedItems[0]][1]==None:
            headertable[SortedItems[0]][1]=FPTree.children[SortedItems[0]]
        else:
            link_same_items(headertable[SortedItems[0]][1], FPTree.children[SortedItems[0]])
    if len(SortedItems)>1:
        updateTree_Table(SortedItems[1:],FPTree.children[SortedItems[0]],headertable,count)

def Build_tree(trans,headertable,count=1):
    recTree=TreeNode('Null',1,None)
    for iii,itemset in enumerate(trans):
        #order the itemset according to the header table
        itemset=[i for i in itemset if i in headertable]
        if len(itemset)>0:
            SortedItems=sorted(list(itemset),key=lambda x:headertable[x][0],reverse=True)
            try:
                updateTree_Table(SortedItems,recTree,headertable,count[iii])
            except:
                updateTree_Table(SortedItems,recTree,headertable,count)
    return recTree,headertable

'''
Mine Tree
'''
def ascendTree(leafNode,PrefixPath):
    if leafNode.parent is not None:
        PrefixPath.append(leafNode.item)
        ascendTree(leafNode.parent, PrefixPath)

def findPrefixPath(baseitem,TreeNode):
    CPB={}
    while TreeNode is not None:
        PrefixPath=[]
        ascendTree(TreeNode,PrefixPath)
        if len(PrefixPath)>1:
            CPB[frozenset(PrefixPath[1:])]=TreeNode.count
        TreeNode=TreeNode.nodelink
    return CPB

def mineTree(inTree, headertable, min_support, preFix, freqItemList):
    reverse_headertable = [v for v,_ in sorted(headertable.items(), key=lambda x: x[1][0])]
    for baseitem in reverse_headertable:
        newFreqSet = preFix.copy()
        newFreqSet.add(baseitem)
        freqItemList[frozenset(newFreqSet)]=0###
        print('newFreqSet:',newFreqSet)
        print('Freqitemls:',freqItemList)

        # Conditional Pattern Base
        CPB = findPrefixPath(baseitem, headertable[baseitem][1])
        freqItemList[frozenset(newFreqSet)]+=sum(CPB.values())

        if len(CPB)!=0:
            print(pd.DataFrame({'Item':[str(baseitem) for i in range(len(CPB))],'Prefix':[(i) for i in CPB],'count':[(i) for i in CPB.values()]}))
        CPBC1=GenC1(CPB,min_support)
        if len(CPBC1)!=0:
            print(CPBC1)
            print('-'*30)
        CondFPTree, Condheadertable = Build_tree(list(CPB.keys()),CPBC1,list(CPB.values()))
        if Condheadertable is not None:
            pass
            #myCondTree.disp(1)
            mineTree(CondFPTree, Condheadertable, min_support, newFreqSet, freqItemList)

if __name__=='__main__':
    simpDat=loadSimpDat()
    min_support=1.9/9*len(simpDat)
    C1=GenC1(simpDat,min_support)
    myTree,myHeader=Build_tree(simpDat,C1)
    freqItemList,preFix={},set([])
    mineTree(myTree, myHeader, min_support, preFix, freqItemList)



newFreqSet: {'I5'}
Freqitemls: {frozenset({'I5'}): 0}
  Item        Prefix  count
0   I5      (I1, I2)      1
1   I5  (I1, I3, I2)      1
{'I1': [2, None], 'I2': [2, None]}
------------------------------
newFreqSet: {'I1', 'I5'}
Freqitemls: {frozenset({'I5'}): 2, frozenset({'I1', 'I5'}): 0}
newFreqSet: {'I5', 'I2'}
Freqitemls: {frozenset({'I5'}): 2, frozenset({'I1', 'I5'}): 0, frozenset({'I5', 'I2'}): 0}
  Item Prefix  count
0   I2   (I1)      2
{'I1': [2, None]}
------------------------------
newFreqSet: {'I1', 'I5', 'I2'}
Freqitemls: {frozenset({'I5'}): 2, frozenset({'I1', 'I5'}): 0, frozenset({'I5', 'I2'}): 2, frozenset({'I1', 'I5', 'I2'}): 0}
newFreqSet: {'I4'}
Freqitemls: {frozenset({'I5'}): 2, frozenset({'I1', 'I5'}): 0, frozenset({'I5', 'I2'}): 2, frozenset({'I1', 'I5', 'I2'}): 0, frozenset({'I4'}): 0}
  Item    Prefix  count
0   I4      (I2)      1
1   I4  (I1, I2)      1
{'I2': [2, None]}
------------------------------
newFreqSet: {'I4', 'I2'}
Freqitemls: {frozenset({'I5'}): 2

In [17]:
A=pd.DataFrame({'FredP':freqItemList}).reset_index()
A['leng']=[len(i) for i in A['index']]
A.sort_values('leng').reset_index(drop=True)

,index,FredP,leng
0,(I5),2,1
1,(I4),2,1
2,(I1),4,1
3,(I3),6,1
4,(I2),0,1
5,"(I1, I5)",0,2
6,"(I5, I2)",2,2
7,"(I4, I2)",0,2
8,"(I1, I2)",0,2
9,"(I3, I2)",2,2


In [18]:
transactions=loadSimpDat()
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df.head()
from mlxtend.frequent_patterns import fpgrowth
FP=fpgrowth(df, min_support=1.9/9,use_colnames=True)
FP['supportnum']=FP['support']*len(transactions)
FP['leng']=[len(i) for i in FP['itemsets']]
FP.sort_values('leng',ascending=True).reset_index(drop=True)

,support,itemsets,supportnum,leng
0,0.777778,(I2),7.0,1
1,0.666667,(I1),6.0,1
2,0.222222,(I5),2.0,1
3,0.222222,(I4),2.0,1
4,0.666667,(I3),6.0,1
5,0.444444,"(I1, I2)",4.0,2
6,0.444444,"(I1, I3)",4.0,2
7,0.222222,"(I1, I5)",2.0,2
8,0.222222,"(I5, I2)",2.0,2
9,0.222222,"(I4, I2)",2.0,2


In [6]:
idx=[]
for m,ii in enumerate([set(i) for i in FP.itemsets]):
    for j in A.FredP:
        if j==(ii):
            idx.append(m)
[k for i,k in enumerate([set(i) for i in FP.itemsets]) if i not in set(idx)]

[{'I2'}, {'I1'}, {'I3'}]